In [37]:
import json

# Demonstration of how to load a file that contains secrets without accidentally leaking those secrets
with open('C:\\Users\\aznma\\Documents\\Github Projects\\DATA320\\credentials.json') as f:
    data = json.load(f)

    # If you want your data to be secure, don't print this variable out!
    # Jupyter will retain a cached version of any printed data and it can be
    # accidentally committed to version control.
    secret_key = data['mongodb']

# We can safely print the length of the secret key. That won't leak any sensitive information.
print(f"My secret key is {len(secret_key)} characters in length.")

My secret key is 73 characters in length.


In [38]:
import pymongo
import certifi

# Connect to the database using known good certificates
client = pymongo.MongoClient(secret_key, tlsCAFile=certifi.where())
print(f"Using MongoDB version {client.server_info()['version']}.")

# Check what databases exist on this server
all_databases = client.list_database_names()
print(f"This MongoDB server has the databases {all_databases}")

# If we know the correct database to talk to, we connect like this:
data320 = client['data320']

# Here is the list of collections within my database
all_collections = data320.list_collection_names()
print(f"This database has the collections {all_collections}")

Using MongoDB version 8.0.15.
This MongoDB server has the databases ['data320', 'admin', 'local']
This database has the collections ['imdb_flawed', 'metacritic']


In [41]:
import pandas as pd

# Retrieve all records from a collection - this can be a large amount of data!
cursor = data320["imdb_flawed"].find()

# Convert this information into a Pandas dataframe
imdb_flawed = pd.DataFrame(cursor)

# Make sure we've read the information correctly
imdb_flawed.head()

,_id,id,title,runtime,user_rating,votes,mpaa_rating,release_date,budget,opening_weekend,gross_sales,genres,cast,director,producer,company
0,68f98887ce51f7d703653539,77631,Grease,110.0,7.2,265183,TV-14::(D),6/13/1978,6000000.0,60759.0,394955690.0,"Comedy, Musical, Romance","John Travolta, Olivia Newton-John, Stockard Ch...",Randal Kleiser,"Allan Carr, Neil A. Machlis, Robert Stigwood","Paramount Pictures, Robert Stigwood Organizati..."
1,68f98887ce51f7d70365353a,78346,Superman,143.0,7.4,172769,TV-PG::(LV),12/10/1978,55000000.0,7465343.0,166200000.0,"Action, Adventure, Sci-Fi","Marlon Brando, Gene Hackman, Christopher Reeve...",Richard Donner,"Charles Greenlaw, Richard Lester, Alexander Sa...","Dovemead Films, Film Export A.G., Internationa..."
2,68f98887ce51f7d70365353b,77416,The Deer Hunter,183.0,8.1,334827,R,12/8/1978,15000000.0,NaN,NaN,"Drama, War","Robert De Niro, John Cazale, John Savage, Chri...",Michael Cimino,"Joann Carelli, Michael Cimino, Michael Deeley,...","EMI Films, Universal Pictures"
3,68f98887ce51f7d70365353c,77651,Halloween,91.0,7.7,267109,TV-14,10/25/1978,300000.0,NaN,70000000.0,"Horror, Thriller","Donald Pleasence, Jamie Lee Curtis, Nancy Kyes...",John Carpenter,"Moustapha Akkad, John Carpenter, Debra Hill, K...","Compass International Pictures, Falcon Interna..."
4,68f98887ce51f7d70365353d,77975,National Lampoon's Animal House,109.0,7.4,119917,"TV-14::(DLSV, TV Rating.)",7/27/1978,3000000.0,201747.0,3371006.0,Comedy,"Tom Hulce, Stephen Furst, Mark Metcalf, Mary L...",John Landis,"Ivan Reitman, Matty Simmons","Universal Pictures, Oregon Film Factory, Stage..."


In [ ]:
import re
import pandas as pd
from tabulate import tabulate

YEAR = 2003

# Transforms data values to visual types
def to_date(s): # Transform into date
    return pd.to_datetime(s, errors="coerce")

def to_minutes(x): # Transform into number
    if pd.isna(x): return pd.NA
    m = re.search(r"(\d+)", str(x))
    return pd.to_numeric(m.group(1), errors="coerce") if m else pd.NA

def to_money(x): # Transform into number too
    if pd.isna(x): return pd.NA
    s = str(x).replace("$", "").replace(",", "").strip()
    return pd.to_numeric(s, errors="coerce")

df = imdb_flawed.copy()
df["release_date"] = df["release_date"].apply(to_date)
df["runtime"] = df["runtime"].apply(to_minutes)
df["budget"] = df["budget"].apply(to_money)
if "title" in df.columns:
    df = df[~df["title"].isna()].copy()

# Fix year
df = df[df["release_date"].dt.year == YEAR].copy()

# Sort data for analysis of budget and runtime values
budget_desc  = df.dropna(subset=["budget"]).sort_values("budget", ascending=False)
budget_asc   = df.dropna(subset=["budget"]).sort_values("budget", ascending=True)
runtime_desc = df.dropna(subset=["runtime"]).sort_values("runtime", ascending=False)
runtime_asc  = df.dropna(subset=["runtime"]).sort_values("runtime", ascending=True)

# Spot check ^^^

# Tabulate helpers
def fmt_money(x):
    try:
        return f"${int(float(x)):,}" if pd.notna(x) else "—"
    except Exception:
        return "—"

def fmt_date(x):
    return x.strftime("%Y-%m-%d") if pd.notna(x) else "—"

def show_table(df_sorted, title, cols=("title","budget","runtime","release_date"), k=10):
    print(f"\n{title}")
    if df_sorted.empty:
        print("(No rows)")
        return
    out = df_sorted.head(k).copy()
    if "budget" in out:
        out["budget"] = out["budget"].apply(fmt_money)
    if "release_date" in out:
        out["release_date"] = out["release_date"].apply(fmt_date)
    print(tabulate(out.loc[:, cols], headers="keys", tablefmt="github", showindex=False))

# Print sorted tables for the chosen year
show_table(budget_desc,  f"Highest Budget ({YEAR}, sorted desc)")
show_table(budget_asc,   f"Lowest Budget ({YEAR}, sorted asc)")
show_table(runtime_desc, f"Longest Runtime ({YEAR}, sorted desc)")
show_table(runtime_asc,  f"Shortest Runtime ({YEAR}, sorted asc)")



Highest Budget (2003, sorted desc)
| title                                                  | budget       |   runtime | release_date   |
|--------------------------------------------------------|--------------|-----------|----------------|
| The Hero: Love Story of a Spy                          | $530,000,000 |       160 | 2003-04-09     |
| LOC: Kargil                                            | $400,000,000 |       255 | 2003-12-12     |
| Koi... Mil Gaya                                        | $350,000,000 |       171 | 2003-08-08     |
| Main Prem Ki Diwani Hoon                               | $300,000,000 |       197 | 2003-06-27     |
| Talaash: The Hunt Begins...                            | $250,000,000 |       153 | 2003-01-03     |
| Terminator 3: Rise of the Machines                     | $200,000,000 |       109 | 2003-06-30     |
| The Matrix Reloaded                                    | $150,000,000 |       138 | 2003-05-07     |
| The Matrix Revolutions             


    Try to think: Why are these movies so high or low?
    Try googling the movies to find more information about them
    Don't guess - try to cite someone else's writing as a reason why this movie might have a high or low budget
    Add a markdown segment at the bottom of your Jupyter notebook that suggests whether you think this data is correct, or whether you discovered a problem during spot checking.

    One movie that caught my eye was The Jungle Book 2 as I grew up watching the first which had a budget of $4 million and runtime of 106 minutes. The first movie was very positive. The second, despite having 5 times the budget had less runtime and was heavily criticized. Additionally, a fellow classmate Ashley Newsom mentioned that the budget does not clarify the currency type which led to many discrepancies being found. One example in my 2003 sample was the movie " Koi... Mil Gaya" which is 350 million INR not USD.

    That said, I don't believe this data to be 100% correct as the analysis of budget sorting is innacurate due to not accounting currency exchanges. Had we did a currency exchange of all data to one, consistent currency type then there could possibly be a proper analysis
